In [1]:
import pandas as pd
import csv
import json
import os
import re
from collections import defaultdict

In [2]:
#do the same but for Android data
json_data_android = []
path = 'Android/'


In [6]:
mag_files, accel_files, gyr_files = [], [], []

In [9]:
def rename_android_files(user):
    path = 'Android/incoming'
    filelist = os.listdir(path) 
    mag_count, accel_count, gyr_count = 1, 1, 1
    #rename all the files to better identify user and data log type
    for f in filelist:
        #get the original filename and its extension
        original_filename, ext = os.path.splitext(f)[0], os.path.splitext(f)[1]
        #See if the file has magnetometer or accelerometer data
        log_type = re.search("[^_]*$", original_filename).group()
        if "mag" in log_type:
            #create the new file name
            new_filename = "%s_magnetometer_%.4d" % (user, mag_count)
            #actually rename the file
            #os.rename(path + f, path + new_filename + ext)
            print new_filename
            #increment mag_count so the next file is sequentially ordered. input order doesnt matter since theyre timestamped
            mag_count += 1
            #add to mag_files list so we can read and clean all magnetometer data later
            #mag_files.append(path + new_filename + ext)
        
        elif "acc" in log_type:
            new_filename = "%s_accelerometer_%.4d" % (user, accel_count)
            #os.rename(path + f, path + new_filename + ext)
            accel_count += 1
            print new_filename
            #accel_files.append(path + new_filename + ext)
        
        elif "gyr" in log_type:
            new_filename = "%s_gyrometer_%.4d" % (user, gyr_count)
            #os.rename(path + f, path + new_filename + ext)
            gyr_count += 1
            print new_filename
            #gyr_files.append(path + new_filename + ext)
        else:
            print "Unhandled File: %s" % (f) #perhaps this should be an exception

In [10]:
#run this function for each user
rename_android_files("Andrew")

Andrew_accelerometer_0001
Andrew_magnetometer_0001
Andrew_gyrometer_0001
Andrew_gyrometer_0002
Andrew_magnetometer_0002
Andrew_gyrometer_0003
Andrew_magnetometer_0003
Andrew_gyrometer_0004
Andrew_gyrometer_0005
Andrew_accelerometer_0002
Andrew_gyrometer_0006
Andrew_gyrometer_0007
Andrew_accelerometer_0003
Andrew_gyrometer_0008
Andrew_accelerometer_0004
Andrew_magnetometer_0004
Andrew_magnetometer_0005
Unhandled File: raw
Andrew_magnetometer_0006
Andrew_accelerometer_0005
Andrew_accelerometer_0006
Andrew_magnetometer_0007
Andrew_gyrometer_0009
Andrew_gyrometer_0010
Andrew_magnetometer_0008
Andrew_accelerometer_0007
Andrew_accelerometer_0008
Andrew_magnetometer_0009
Andrew_accelerometer_0009
Andrew_magnetometer_0010


In [95]:
def read_in_android_accelerometer(fp):
    with open("Android/Andrew_accelerometer_0001.log", "r+") as datafile:
            reader = csv.reader(datafile, delimiter = ";")
            header = next(reader)
            for row in reader:
                #some accelometer readings don't have GPS data, handle them like this:
                if len(row) == 7:
                    observation = {'device': "Android", '_user_id': "test_user", 
                                   'timestamp': re.sub("_", "T", row[1][0:-4]),
                               'data': {'x_accel': row[3], 'y_accel': row[4], 'z_accel': row[3]}}
                #some accelometer readings do have GPS data, handle them like this:
                elif len(row) == 11: 
                    observation = {'device': "Android", '_user_id': "test_user", 
                                   'timestamp': re.sub("_", "T", row[1][0:-4]),
                               'data': {'latitude': row[3], 'longitude': row[4], 'relative_altitude': row[5],
                                        'accuracy': row[6], 'x_accel': row[7], 'y_accel': row[8], 'z_accel': row[9]}}
                json_data_android.append(observation)           


In [96]:
path = 'Android/'

def get_all_android_files():
    filelist = os.listdir(path) 
    for file in filelist:
        fp = path + file
        if 'accelerometer' in fp:
            read_in_android_accelerometer(fp)

In [97]:
get_all_android_files()        

In [98]:
androidFull = pd.io.json.json_normalize(json_data_android).set_index('timestamp')
androidFull.index = pd.to_datetime(androidFull.index)
json_dataAndroid = json.dumps(json_data_android)

In [100]:
androidFull.head()

,_user_id,data.x_accel,data.y_accel,data.z_accel,device
timestamp,,,,,
2017-03-10 16:00:00,test_user,-0.953,1.854,-0.953,Android
2017-03-10 16:00:00,test_user,-0.953,1.854,-0.953,Android
2017-03-11 03:00:00,test_user,-1.717,2.247,-1.717,Android
2017-03-11 03:00:00,test_user,-1.717,2.247,-1.717,Android
2017-03-11 04:00:00,test_user,-1.717,2.247,-1.717,Android
